In [1]:
import config
import os, numpy as np, pandas, sklearn
import mido
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
## NN libs
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.preprocessing import RobustScaler
import keras
from keras import regularizers
from keras.utils import to_categorical
from keras.models import Sequential
from keras.optimizers import SGD, Adam
from keras.layers import Input, Dense, Activation, Conv1D, Conv2D, Dropout, Flatten
from keras.layers import Conv2DTranspose, Reshape, MaxPooling2D, UpSampling2D, UpSampling1D, MaxPooling1D
from keras.layers import LocallyConnected1D, LocallyConnected2D
from keras.layers import Input, LSTM, RepeatVector
from keras.models import Model
from keras.callbacks import TensorBoard

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# local libs
import config, models, functions
from data import data, midi, midi_generators as g
from utils import io, models_io

 Context :: namedtuple(
[ max_t = float
, dt = float
, n_instances = int
, note_length = int
, bpm = float
, tempo = float
, ticks_per_beat = int
]



In [4]:
context = data.init()

Setting up params

 >> Context(max_t=10.0, dt=0.01, n_instances=1000, note_length=0.03, bpm=120.0, tempo=500000, ticks_per_beat=480)


In [5]:
# fn = (config.dataset_dir + '4-floor-120bpm.mid')
# mid = io.import_midifile(fn)

## Setup a generator

1. What is the least amount of 'hidden' nodes needed to learn a straight rythm (e.g. 8th notes with different frequencies)
2. Can we create a model of a generic function sin(2 pi f t + phase)
    - using x: t -> y: sin(2p t)
    - using x: [f, t, phase] -> y: sin(2p f t + phase)
    - using x: sin([t1, t2, t3) -> y: [f, t, phase]
        - such a model should be able to learn complex patterns, such as sin(f1+p1) + sin(f2+p2) + sin(f3+p3)

In [6]:
n = 100
x_train = g.gen_data(context, n)
# x_train = np.zeros([10,100,127])
y_train = x_train

[DEBUG] >
 |>  WARNING
 |>  type not == 0
[DEBUG] >
 |>  WARNING
 |>  type not == 0
[DEBUG] >
 |>  WARNING
 |>  type not == 0
[DEBUG] >
 |>  WARNING
 |>  type not == 0
[DEBUG] >
 |>  WARNING
 |>  type not == 0
[DEBUG] >
 |>  WARNING
 |>  type not == 0
[DEBUG] >
 |>  WARNING
 |>  type not == 0
[DEBUG] >
 |>  WARNING
 |>  type not == 0
[DEBUG] >
 |>  WARNING
 |>  type not == 0
[DEBUG] >
 |>  WARNING
 |>  type not == 0
[DEBUG] >
 |>  WARNING
 |>  type not == 0
[DEBUG] >
 |>  WARNING
 |>  type not == 0
[DEBUG] >
 |>  WARNING
 |>  type not == 0
[DEBUG] >
 |>  WARNING
 |>  type not == 0
[DEBUG] >
 |>  WARNING
 |>  type not == 0
[DEBUG] >
 |>  WARNING
 |>  type not == 0
[DEBUG] >
 |>  WARNING
 |>  type not == 0
[DEBUG] >
 |>  WARNING
 |>  type not == 0
[DEBUG] >
 |>  WARNING
 |>  type not == 0
[DEBUG] >
 |>  WARNING
 |>  type not == 0
[DEBUG] >
 |>  WARNING
 |>  type not == 0
[DEBUG] >
 |>  WARNING
 |>  type not == 0
[DEBUG] >
 |>  WARNING
 |>  type not == 0
[DEBUG] >
 |>  WARNING
 |>  type n

In [7]:
y_train = x_train
n_samples = x_train[0]
input_shape = x_train.shape[1:] # shape of a single sample
output_shape = y_train.shape[1:]
# output_length = y_train.shape[1]
# output_length = (y_train[0]).shape[0] # = length of an individual label
hidden_layer_length = 100
x_train.shape

(100, 1000, 127)

In [8]:
def resolution_reducer(input_shape, amt=2):
    input_layer = Input(shape=input_shape)
    x = input_layer
    x = MaxPooling1D(pool_size=amt, strides=amt)(x)
    model = Model(inputs=input_layer, outputs=x)
    return model

reducer = resolution_reducer(input_shape,2)
reducer.summary()
# reducer.compile('adam',loss='mse')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000, 127)         0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 500, 127)          0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [9]:
reducer.get_config()

{'input_layers': [['input_1', 0, 0]],
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 1000, 127),
    'dtype': 'float32',
    'name': 'input_1',
    'sparse': False},
   'inbound_nodes': [],
   'name': 'input_1'},
  {'class_name': 'MaxPooling1D',
   'config': {'name': 'max_pooling1d_1',
    'padding': 'valid',
    'pool_size': (2,),
    'strides': (2,),
    'trainable': True},
   'inbound_nodes': [[['input_1', 0, 0, {}]]],
   'name': 'max_pooling1d_1'}],
 'name': 'model_1',
 'output_layers': [['max_pooling1d_1', 0, 0]]}

In [10]:
reducer.output_shape

(None, 500, 127)

In [32]:
# functional syntax: lambda x: lambda y: z
def encoder(input_shape, output_length, dropout=0.10):
    input_layer = Input(shape=input_shape)
    x = input_layer
    x = Flatten()(x)
#     x = Dense(output_length * 2, activation='relu')(x)
#     x = Dense(output_length * 2, activation='relu')(x)
    x = Dense(output_length, activation='sigmoid')(x)
#     x = Dense(output_length, activation='relu', activity_regularizer=regularizers.l1(10e-5))(x)
    #     model.add(Dropout(dropout))    
    model = Model(inputs=input_layer, outputs=x)
    return model


def decoder(input_length, output_shape, dropout=0.10):
    input_layer = Input(shape=(input_length,))
    x = input_layer
    shape = (10,100) # 1 additional dimension
    x = Dense(np.prod(shape), activation='relu')(x) # 4*4*8 = 128
    x = Reshape(shape)(x)
    x = UpSampling1D(10)(x)
    x = Conv1D(100, 2, strides=2, activation='relu')(x) # 50,100
    x = UpSampling1D(output_shape[0] / 50)(x)
    x = Dense(output_shape[1], activation='relu')(x)
#     x = Dense(output_shape[1], activation='relu')(x)
#     x = Dense(output_shape[1], activation='relu')(x)
    #     x = LocallyConnected1D(output_shape[1], kernel_size=1, activation='relu')(x)
    # x = Dense(output_length, activation='softmax')(x)
    model = Model(inputs=input_layer, outputs=x)
    #     model.add(Dropout(dropout))
    return model


def autoencoder(input_shape, output_shape, hidden_layer_length=10, dropout=0.10, verbose=False):
    input_ = Input(shape=input_shape)
    reducer = resolution_reducer(input_shape,2)
    if verbose:
        reducer.summary()
    input_shape = reducer.output_shape[1:]
    encode = encoder(input_shape, hidden_layer_length, dropout)
    if verbose:
        encode.summary()
    decode = decoder(hidden_layer_length, output_shape, dropout)
    if verbose:
        encode.summary()
    model = Model(input_, decode(encode(reducer(input_))))
    return encode, decode, reducer, model


dropout = 0.
encoder, decoder, reducer, auto = autoencoder(input_shape, output_shape, hidden_layer_length, dropout, True)
# summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 1000, 127)         0         
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 500, 127)          0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 500, 127)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 63500)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)               6350100   
Total params: 6,350,100
Trainable params: 6,350,100
Non-trainable params: 0
______

In [33]:
# def decoder(input_length, output_shape, dropout=0.10):
#     input_layer = Input(shape=(input_length,))
#     x = input_layer
#     shape = (10,100) # 1 additional dimension
#     x = Dense(np.prod(shape), activation='relu')(x) # 4*4*8 = 128
#     x = Reshape(shape)(x)
#     x = UpSampling1D(10)(x)
#     x = Conv1D(100, 2, strides=2, activation='relu')(x) # 50,100
#     x = UpSampling1D(output_shape[0] / 50)(x)
#     x = Dense(output_shape[1], activation='relu')(x)
# #     x = Dense(output_shape[1], activation='relu')(x)
# #     x = Dense(output_shape[1], activation='relu')(x)
#     #     x = LocallyConnected1D(output_shape[1], kernel_size=1, activation='relu')(x)
#     # x = Dense(output_length, activation='softmax')(x)
#     model = Model(inputs=input_layer, outputs=x)
#     #     model.add(Dropout(dropout))
#     return model, model.summary

In [34]:
metrics = ['accuracy','mse','categorical_crossentropy']
loss = 'binary_crossentropy'# binary_crossentropy categorical_crossentropy
optimizer = 'adadelta'
auto.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [35]:
batch_size = 16
# n epochs = n iterations over all the training data
epochs = 15

In [36]:
auto.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, callbacks=[TensorBoard(log_dir=config.tmp_log_dir)])
# model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,
#           validation_split=1/6, callbacks=[TensorBoard(log_dir=config.tmp_model_dir)])

Epoch 1/15
100/100 [==============================] - 3s 33ms/step - loss: 0.0264 - acc: 1.0000 - mean_squared_error: 0.0024 - categorical_crossentropy: 0.0645
Epoch 2/15
100/100 [==============================] - 3s 28ms/step - loss: 0.0052 - acc: 1.0000 - mean_squared_error: 2.2110e-04 - categorical_crossentropy: 0.0516
Epoch 3/15
100/100 [==============================] - 3s 28ms/step - loss: 0.0012 - acc: 1.0000 - mean_squared_error: 5.7512e-05 - categorical_crossentropy: nan
Epoch 4/15
100/100 [==============================] - 3s 26ms/step - loss: 5.0139e-04 - acc: 1.0000 - mean_squared_error: 4.1070e-05 - categorical_crossentropy: nan
Epoch 5/15
100/100 [==============================] - 3s 28ms/step - loss: 4.2173e-04 - acc: 1.0000 - mean_squared_error: 3.9795e-05 - categorical_crossentropy: nan
Epoch 6/15
100/100 [==============================] - 3s 27ms/step - loss: 3.9542e-04 - acc: 1.0000 - mean_squared_error: 3.9142e-05 - categorical_crossentropy: nan
Epoch 7/15
100/100 [

In [16]:
mid1 = midi.encode(context, g.render_midi(context, f=2))
mid2 = midi.encode(context, g.render_midi(context, f=3))
mid1.shape

[DEBUG] >
 |>  WARNING
 |>  type not == 0
[DEBUG] >
 |>  WARNING
 |>  type not == 0


(1000, 127)

In [38]:
x_test = np.stack([mid1, mid2])
latent = encoder.predict(reducer.predict(x_test))
latent.shape

(2, 100)

In [23]:
latent.shape

(2, 100)

In [29]:
maxx = np.maximum(latent[0],latent[1])
maxx.shape

(100,)

In [31]:
latent[0].shape

(100,)

In [52]:
arrays = decoder.predict(np.stack([latent[0]]))
arrays.shape

(1, 1000, 127)

In [53]:
mid = midi.decode_track(context, arrays[0])

[INFO] :
 |  len
 |  9.313541666666707
 |  10.0


In [54]:
io.export_midifile(mid, config.dataset_dir + 'song_export-cycles-mod.mid')